In [1]:
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def load_files(path):
    '''
    Input: path to parent child folder
    Output: paths to parent files, paths to child files    
    '''
    parent_files = []
    child_files = []
    for file in os.listdir(path):
        if file == "Thumbs.db":
            continue
        if file.split("_")[2][0] == "1":
            parent_files.append(os.path.join(path, file))
        elif file.split("_")[2][0] == "2":
            child_files.append(os.path.join(path, file))
    return parent_files, child_files
# print(load_files("C:\\Users\\Manmeet\\Desktop\\Spring 2019\\NN\\Project\\KinFaceW-II\\KinFaceW-II\\images\\father-dau"))

In [0]:
TRAIN_ROOT_DIR = "C:\\Users\\Manmeet\\Desktop\\Spring 2019\\NN\\Project\\KinFaceW-II\\KinFaceW-II\\images\\"
TEST_ROOT_DIR = "C:\\Users\\Manmeet\\Desktop\\Spring 2019\\NN\\Project\\KinFaceW-I\\KinFaceW-I\\images\\"

def make_pair_dict(ROOT_DIR):
        pairs = os.listdir(ROOT_DIR)

        parent_dict = {}
        child_dict = {}

        comb_key = 1
        for pair in pairs:
            pair_path = os.path.join(ROOT_DIR, pair)
            parent_dict[comb_key], child_dict[comb_key] = load_files(pair_path)
            comb_key += 1
        return parent_dict, child_dict
    
train_parent_dict, train_child_dict = make_pair_dict(TRAIN_ROOT_DIR)
test_parent_dict, test_child_dict = make_pair_dict(TEST_ROOT_DIR)

In [0]:
def make_pairs(parent_dict, child_dict):
    matched_pairs = []
    unmatched_pairs = []
    for parent_key in parent_dict.keys():
        for child_key in child_dict.keys():
            if parent_key == child_key:
                for file in zip(parent_dict[parent_key], child_dict[child_key]):
                    matched_pairs.append(load_images(file))
            else:
                for file in zip(parent_dict[parent_key], child_dict[child_key]):
                    unmatched_pairs.append(load_images(file))
    return matched_pairs, unmatched_pairs
                    
def load_images(file):
    images = np.zeros((64, 64, 6))
    parent = plt.imread(file[0])
    images[:,:,0:3] = parent
    child = plt.imread(file[1])
    images[:,:,3:6] = child
    return images

train_matched_pairs, train_unmatched_pairs = make_pairs(train_parent_dict, train_child_dict)
test_matched_pairs, test_unmatched_pairs = make_pairs(test_parent_dict, test_child_dict)

In [0]:
import numpy as np
train_matched_pairs = np.load("/content/drive/My Drive/train_matched_pairs.npy")
train_unmatched_pairs = np.load("/content/drive/My Drive/train_unmatched_pairs.npy")
test_matched_pairs = np.load("/content/drive/My Drive/test_matched_pairs.npy")
test_unmatched_pairs = np.load("/content/drive/My Drive/test_unmatched_pairs.npy")

In [3]:
from keras.utils.np_utils import to_categorical
def train_test_and_make_labels(matched_pairs, unmatched_pairs):
    matched_pairs = np.array(matched_pairs)
    unmatched_pairs = np.array(unmatched_pairs)
    labels = to_categorical(np.r_[np.ones(matched_pairs.shape[0]), np.zeros(unmatched_pairs.shape[0])])
    pairs = np.r_[matched_pairs, unmatched_pairs]
    return pairs, labels

X_train, Y_train = train_test_and_make_labels(train_matched_pairs, train_unmatched_pairs)
X_test, Y_test = train_test_and_make_labels(test_matched_pairs, test_unmatched_pairs)

Using TensorFlow backend.


In [5]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((4000, 64, 64, 6), (4000, 2), (2005, 64, 64, 6), (2005, 2))

In [0]:
from keras import layers
from keras import models
from keras import optimizers

def model_2():
    model = models.Sequential()
    model.add(layers.Conv2D(16, (5, 5), strides=1, padding='valid', input_shape=(64, 64, 6), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(32, (5, 5), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (5, 5), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (2, 2), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('tanh'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(512, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('tanh'))
    
    # model.add(layers.Dropout(0.4))
    model.add(layers.Dense(2, activation='softmax'))
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [7]:
model = model_2()
model.fit(X_train/255, Y_train, steps_per_epoch=20, epochs=5, verbose=1, shuffle=True, validation_data=(X_test/255, Y_test), validation_steps=100)

Train on 4000 samples, validate on 2005 samples
Epoch 1/5
20/20 [==============================] - 30s 2s/step - loss: 0.4571 - acc: 0.7841 - val_loss: 0.8970 - val_acc: 0.5751
Epoch 2/5
20/20 [==============================] - 29s 1s/step - loss: 0.2234 - acc: 0.9192 - val_loss: 0.6294 - val_acc: 0.7531
Epoch 3/5
20/20 [==============================] - 29s 1s/step - loss: 0.0960 - acc: 0.9801 - val_loss: 0.7743 - val_acc: 0.7337
Epoch 4/5
20/20 [==============================] - 29s 1s/step - loss: 0.1049 - acc: 0.9699 - val_loss: 0.9105 - val_acc: 0.7835
Epoch 5/5
20/20 [==============================] - 29s 1s/step - loss: 0.0280 - acc: 0.9989 - val_loss: 1.0015 - val_acc: 0.7805


In [0]:
model.save("2-classifier-model.h5")